# XGBoost
La técnica de modelado más precisa para datos estructurados.

En este tutorial, aprenderá cómo crear y optimizar modelos con aumento de gradiente. Este método domina muchas competiciones de Kaggle y logra resultados de última generación en una variedad de conjuntos de datos.

### Introducción
Durante gran parte de este curso, ha realizado predicciones con el método de bosque aleatorio, que logra un mejor rendimiento que un único árbol de decisión simplemente promediando las predicciones de muchos árboles de decisión.

Nos referimos al método del bosque aleatorio como un "método de conjunto". Por definición, los métodos de conjunto combinan las predicciones de varios modelos (por ejemplo, varios árboles, en el caso de bosques aleatorios).

A continuación, aprenderemos sobre otro método de conjunto llamado aumento de gradiente.

### Aumento de gradiente
El aumento de gradiente es un método que pasa por ciclos para agregar modelos de forma iterativa a un conjunto.

Comienza inicializando el conjunto con un único modelo, cuyas predicciones pueden ser bastante ingenuas. (Incluso si sus predicciones son tremendamente inexactas, las adiciones posteriores al conjunto abordarán esos errores).

Luego, iniciamos el ciclo:

. Primero, utilizamos el conjunto actual para generar predicciones para cada observación en el conjunto de datos. Para hacer una predicción, sumamos las predicciones de todos los modelos del conjunto.

. Estas predicciones se utilizan para calcular una función de pérdida (como el error cuadrático medio, por ejemplo).

. Luego, usamos la función de pérdida para ajustar un nuevo modelo que se agregará al conjunto.  Específicamente, determinamos los parámetros del modelo para que agregar este nuevo modelo al conjunto reduzca la pérdida. (Nota al margen: el "gradiente" en "aumento de gradiente" se refiere al hecho de que usaremos el descenso de gradiente en la función de pérdida para determinar los parámetros en este nuevo modelo).

. Finalmente, agregamos el nuevo modelo al conjunto, y...

. ... ¡repetir!

### Ejemplo
Comenzamos cargando los datos de entrenamiento y validación en X_train, X_valid, y_train e y_valid.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

# Separate data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

En este ejemplo, trabajará con la biblioteca XGBoost. XGBoost significa aumento de gradiente extremo, que es una implementación de aumento de gradiente con varias características adicionales centradas en el rendimiento y la velocidad. (Scikit-learn tiene otra versión de aumento de gradiente, pero XGBoost tiene algunas ventajas técnicas).

En la siguiente celda de código, importamos la API scikit-learn para XGBoost (xgboost.XGBRegressor). Esto nos permite construir y ajustar un modelo tal como lo haríamos en scikit-learn. Como verá en el resultado, la clase XGBRegressor tiene muchos parámetros ajustables; ¡pronto aprenderá sobre ellos!

In [3]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

También hacemos predicciones y evaluamos el modelo.

In [4]:
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

Mean Absolute Error: 233413.53307023196


### Ajuste de parámetros
XGBoost tiene algunos parámetros que pueden afectar drásticamente la precisión y la velocidad del entrenamiento. Los primeros parámetros que debes entender son:

n_estimadores
n_estimators especifica cuántas veces se debe pasar por el ciclo de modelado descrito anteriormente. Es igual al número de modelos que incluimos en el conjunto.

Un valor demasiado bajo provoca un ajuste insuficiente, lo que conduce a predicciones inexactas tanto en los datos de entrenamiento como en los de prueba.
Un valor demasiado alto provoca un sobreajuste, lo que provoca predicciones precisas en los datos de entrenamiento, pero predicciones inexactas en los datos de prueba (que es lo que nos importa).
Los valores típicos oscilan entre 100 y 1000, aunque esto depende en gran medida del parámetro learning_rate que se analiza a continuación.

Aquí está el código para establecer el número de modelos en el conjunto:

In [5]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

rondas_paradas_tempranas

early_stopping_rounds ofrece una manera de encontrar automáticamente el valor ideal para n_estimators. La parada anticipada hace que el modelo deje de iterar cuando la puntuación de validación deja de mejorar, incluso si no estamos en la parada brusca de n_estimators. Es inteligente establecer un valor alto para n_estimators y luego usar early_stopping_rounds para encontrar el momento óptimo para dejar de iterar.

Dado que el azar a veces provoca una sola ronda en la que las puntuaciones de validación no mejoran, es necesario especificar un número de rondas de deterioro directo que se permitirán antes de detenerse. Establecer early_stopping_rounds=5 es una opción razonable. En este caso, nos detenemos después de 5 rondas seguidas de deterioro de las puntuaciones de validación.

Cuando utilice early_stopping_rounds, también debe reservar algunos datos para calcular las puntuaciones de validación; esto se hace configurando el parámetro eval_set.

Podemos modificar el ejemplo anterior para incluir la parada anticipada:

In [6]:
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)],
             verbose=False)

c:\Users\karlo\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

Si más adelante desea ajustar un modelo con todos sus datos, establezca n_estimators en cualquier valor que considere óptimo cuando se ejecute con parada anticipada.

tasa de aprendizaje
En lugar de obtener predicciones simplemente sumando las predicciones de cada modelo componente, podemos multiplicar las predicciones de cada modelo por un número pequeño (conocido como tasa de aprendizaje) antes de agregarlas.

Esto significa que cada árbol que agregamos al conjunto nos ayuda menos. Entonces, podemos establecer un valor más alto para n_estimators sin sobreajustar. Si utilizamos la parada anticipada, el número apropiado de árboles se determinará automáticamente.

En general, una tasa de aprendizaje pequeña y una gran cantidad de estimadores producirán modelos XGBoost más precisos, aunque también llevará más tiempo entrenar el modelo, ya que realiza más iteraciones a lo largo del ciclo. Por defecto, XGBoost establece learning_rate=0.1.

Modificar el ejemplo anterior para cambiar la tasa de aprendizaje produce el siguiente código:

In [7]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

n_trabajos

En conjuntos de datos más grandes donde se considera el tiempo de ejecución, puede utilizar el paralelismo para construir sus modelos más rápido. Es común establecer el parámetro n_jobs igual al número de núcleos de su máquina. En conjuntos de datos más pequeños, esto no ayudará.

El modelo resultante no será mejor, por lo que la microoptimización para el tiempo de adaptación no suele ser más que una distracción. Sin embargo, es útil en conjuntos de datos grandes donde, de otro modo, pasaría mucho tiempo esperando durante el comando de ajuste.

Aquí está el ejemplo modificado:

In [8]:
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=4,
             num_parallel_tree=None, random_state=None, ...)

### Conclusión

XGBoost es una biblioteca de software líder para trabajar con datos tabulares estándar (el tipo de datos que almacena en Pandas DataFrames, a diferencia de tipos de datos más exóticos como imágenes y videos). Con un ajuste cuidadoso de los parámetros, puede entrenar modelos de alta precisión.

# EJERCICIOS

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = low_cardinality_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

### Paso 1: construir modelo
#### Parte A
En este paso, creará y entrenará su primer modelo con aumento de gradiente.

Comience configurando my_model_1 en un modelo XGBoost. Utilice la clase XGBRegressor y establezca la semilla aleatoria en 0 (random_state=0). Deje todos los demás parámetros por defecto.
Luego, ajuste el modelo a los datos de entrenamiento en X_train e y_train.

In [ ]:
from xgboost import XGBRegressor

# Define the model
my_model_1 = XGBRegressor(random_state=0) # Your code here

# Fit the model
my_model_1.fit(X_train, y_train) # Your code here

#### Parte B
Establezca predicciones_1 para las predicciones del modelo para los datos de validación. Recuerde que las funciones de validación se almacenan en X_valid.

In [ ]:
from sklearn.metrics import mean_absolute_error

# Get predictions
predictions_1 = my_model_1.predict(X_valid) # Your code here

### Parte C
Finalmente, use la función mean_absolute_error() para calcular el error absoluto medio (MAE) correspondiente a las predicciones para el conjunto de validación. Recuerde que las etiquetas de los datos de validación se almacenan en y_valid.

In [ ]:
# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

### Paso 2: mejorar el modelo
Ahora que ha entrenado un modelo predeterminado como punto de referencia, es hora de modificar los parámetros para ver si puede obtener un mejor rendimiento.

Comience configurando my_model_2 en un modelo XGBoost, usando la clase XGBRegressor. Utilice lo que aprendió en el tutorial anterior para descubrir cómo cambiar los parámetros predeterminados (como n_estimators y learning_rate) para obtener mejores resultados.
Luego, ajuste el modelo a los datos de entrenamiento en X_train e y_train.
Establezca predicciones_2 para las predicciones del modelo para los datos de validación. Recuerde que las funciones de validación se almacenan en X_valid.
Finalmente, use la función mean_absolute_error() para calcular el error absoluto medio (MAE) correspondiente a las predicciones en el conjunto de validación. Recuerde que las etiquetas de los datos de validación se almacenan en y_valid.
Para que este paso se marque como correcto, su modelo en my_model_2 debe alcanzar un MAE más bajo que el modelo en my_model_1.

In [ ]:
# Define the model
my_model_2 = XGBRegressor(n_estimators=1000, learning_rate=0.05) # Your code here

# Fit the model
my_model_2.fit(X_train, y_train) # Your code here

# Get predictions
predictions_2 = my_model_2.predict(X_valid) # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2, y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

### Paso 3: romper el modelo
En este paso, creará un modelo que funciona peor que el modelo original del Paso 1. Esto le ayudará a desarrollar su intuición sobre cómo establecer parámetros. Incluso podría descubrir que accidentalmente obtiene un mejor rendimiento, lo cual, en última instancia, es un problema agradable y una valiosa experiencia de aprendizaje.

. Comience configurando my_model_3 en un modelo XGBoost, usando la clase XGBRegressor. Utilice lo que aprendió en el tutorial anterior para descubrir cómo cambiar los parámetros predeterminados (como n_estimators y learning_rate) para diseñar un modelo que obtenga un MAE alto.

. Luego, ajuste el modelo a los datos de entrenamiento en X_train e y_train.

. Establezca predicciones_3 para las predicciones del modelo para los datos de validación. Recuerde que las funciones de validación se almacenan en X_valid.

. Finalmente, use la función mean_absolute_error() para calcular el error absoluto medio (MAE) correspondiente a las predicciones en el conjunto de validación. Recuerde que las etiquetas de los datos de validación se almacenan en y_valid.

Para que este paso se marque como correcto, su modelo en my_model_3 debe alcanzar un MAE más alto que el modelo en my_model_1.

In [ ]:
# Define the model
my_model_3 = XGBRegressor(n_estimators=1)

# Fit the model
my_model_3.fit(X_train, y_train) # Your code here

# Get predictions
predictions_3 = my_model_3.predict(X_valid)

# Calculate MAE
mae_3 = mean_absolute_error(predictions_3, y_valid)

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_3)